<a href="https://colab.research.google.com/github/vignesh99/CS6300-Speech-Technology/blob/master/E2E_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
                                    #Get data files onto root
%cd
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/TIMIT\ E2E/timit.zip /root
!unzip /root/timit.zip -d /root
!ls

Streaming output truncated to the last 5000 lines.
  inflating: /root/timit_train/mgar0/sx132.mfc  
  inflating: /root/timit_train/mgar0/sa1.wrd  
  inflating: /root/timit_train/mgar0/sa2.phn  
  inflating: /root/timit_train/mgar0/sx132.PHN_SP  
  inflating: /root/timit_train/mgar0/sa2.wav  
  inflating: /root/timit_train/mgar0/sx312.wrd  
  inflating: /root/timit_train/mgar0/si1694.wav  
 extracting: /root/timit_train/mgar0/sx222.txt  
  inflating: /root/timit_train/mgar0/sx222.phn  
  inflating: /root/timit_train/mgar0/sx42.wav  
  inflating: /root/timit_train/mgar0/si1842.txt  
  inflating: /root/timit_train/mgar0/sa2.mfc  
  inflating: /root/timit_train/mgar0/sx132.wrd  
  inflating: /root/timit_train/mgar0/si1842.phn  
  inflating: /root/timit_train/mgar0/sx42.wrd  
  inflating: /root/timit_train/mgar0/si1212.phn  
  inflating: /root/timit_train/mgar0/si1212.wav  
  inflating: /root/timit_train/mgar0/si1694.wrd  
  inflating: /root/timit_train/mpsw0/sx257.phn  
  inflating: /root/

In [46]:
                                    #Import libraries
from pylab import *
import os
from google.colab import files

In [44]:
                                    #Use pkl to store model
!pip install pickle-mixin
from pickle import dump
from pickle import load

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-cp36-none-any.whl size=6000 sha256=f8ca344267c6d697de3b21ba1fef9a9cd86df678cfd3af4bde925a045b12490b
  Stored in directory: /root/.cache/pip/wheels/cd/05/42/71de70fa36b9cbb7657bb5793a16f8028c1cdc1bdd3b8e1ac3
Successfully built pickle-mixin


In [41]:
def mostrepeatedwords(Nrep=154) :
    allwords = []
                                    #Go through all folders and consolidate the words
    for subdir, dirs, files in os.walk("timit_train"):
        dirname = subdir.split(os.path.sep)[-1]
        print('Directory:', dirname)
        for f in files:
            if f.endswith('.txt'):
                                    #Get words of each sentence from .txt file
                !cp -avr /root/timit_train/$dirname/$f /root
                words = np.genfromtxt(f,delimiter=' ',dtype=str)
                words = words[2:]
                words[-1] = words[-1][:-1]
                                    #Put all the words in a list
                allwords += list(map(str.lower,list(words)))
                !rm $f
                                    #Get the unique set of words and their counts
    wordunique,wordcount = np.unique(allwords,return_counts=True)
                                    #Choose only those words with significant repitions
    indrep = np.where(wordcount >= Nrep)[0]
    #print(wordcount)
    len(indrep)
    wordclass = wordunique[indrep]
    return wordclass,wordunique,wordcount,indrep

In [31]:
words,wordset,reps,inds = mostrepeatedwords()
words

Streaming output truncated to the last 5000 lines.
'/root/timit_train/msds0/sx357.txt' -> '/root/sx357.txt'
'/root/timit_train/msds0/si1707.txt' -> '/root/si1707.txt'
'/root/timit_train/msds0/sa1.txt' -> '/root/sa1.txt'
'/root/timit_train/msds0/sx267.txt' -> '/root/sx267.txt'
'/root/timit_train/msds0/sx87.txt' -> '/root/sx87.txt'
Directory: mtjs0
'/root/timit_train/mtjs0/sa2.txt' -> '/root/sa2.txt'
'/root/timit_train/mtjs0/sx292.txt' -> '/root/sx292.txt'
'/root/timit_train/mtjs0/si562.txt' -> '/root/si562.txt'
'/root/timit_train/mtjs0/sx202.txt' -> '/root/sx202.txt'
'/root/timit_train/mtjs0/si1822.txt' -> '/root/si1822.txt'
'/root/timit_train/mtjs0/sx22.txt' -> '/root/sx22.txt'
'/root/timit_train/mtjs0/sx382.txt' -> '/root/sx382.txt'
'/root/timit_train/mtjs0/si1192.txt' -> '/root/si1192.txt'
'/root/timit_train/mtjs0/sa1.txt' -> '/root/sa1.txt'
'/root/timit_train/mtjs0/sx112.txt' -> '/root/sx112.txt'
Directory: mrdm0
'/root/timit_train/mrdm0/sa2.txt' -> '/root/sa2.txt'
'/root/timit_trai

array(['a', 'all', 'an', 'and', 'are', 'as', 'ask', 'be', 'but', 'by',
       'carry', 'dark', "don't", 'for', 'from', 'greasy', 'had', 'have',
       'he', 'her', 'his', 'in', 'is', 'it', 'like', 'me', 'not', 'of',
       'oily', 'on', 'rag', 'she', 'suit', 'that', 'the', 'they', 'this',
       'to', 'was', 'wash', 'water', 'we', 'with', 'year', 'you', 'your'],
      dtype='<U18')

In [51]:
inds = np.where(reps >= 154)[0]
words = wordset[inds]
print(len(inds))
print(list(words))

36
['a', 'all', 'an', 'and', 'are', 'ask', 'be', 'carry', 'dark', "don't", 'for', 'greasy', 'had', 'he', 'his', 'in', 'is', 'like', 'me', 'of', 'oily', 'on', 'rag', 'she', 'suit', 'that', 'the', 'to', 'was', 'wash', 'water', 'we', 'with', 'year', 'you', 'your']


In [47]:
                                    #Dump words model file
dump(words, open('words.pkl', 'wb'))
dump(inds, open('inds.pkl', 'wb'))
dump(reps, open('reps.pkl', 'wb'))
dump(wordset, open('wordset.pkl', 'wb'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
                                    #Download model files
files.download("/root/words.pkl")
files.download("/root/inds.pkl")
files.download("/root/reps.pkl")
files.download("/root/wordset.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>